#Finetuning LLM model

- The notebook is optimized for execution on Google Colab

- Unsloth was utilized for finetuning to expedite the training process. Specifically, the "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" configuration was employed to leverage its capabilities for accelerated training.

- Llama 3 served as the Language Model (LLM) of choice, primarily due to its accessibility and performance. The model, identified as "unsloth/llama-3-8b-bnb-4bit", offered robust features tailored to the task requirements.

- The dataset which had 50k+ entries underwent an 80/20 train-test split. However, validation during training was omitted due to hardware resource constraints, which limited the feasibility of additional validation steps.

- Testing was conducted manually using randomly selected prompts and responses from the test dataset.

- Stop words were not removed during the data preprocessing stage, as it was determined that their exclusion did not significantly impact the model's performance for this particular task.

# Import libraries

In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

# Load model


In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.




---



# Integrating LoRA adapters

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Data preparation


In [4]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset

# system prompt
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Set token
EOS_TOKEN = tokenizer.eos_token

# Function to formating prompts
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

# Load dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
formatted_dataset = dataset.map(formatting_prompts_func, batched = True,)

# Convert dataset to pandas DataFrame
data = dataset.to_pandas()

# Split dataset into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Convert back to datasets
train_dataset = dataset.from_pandas(train_data)
test_dataset = dataset.from_pandas(test_data)

# Formate datasets
train_dataset = train_dataset.map(formatting_prompts_func, batched = True,)
test_dataset = test_dataset.map(formatting_prompts_func, batched = True,)

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/41408 [00:00<?, ? examples/s]

Map:   0%|          | 0/10352 [00:00<?, ? examples/s]

In [5]:
# View train dataset
train_dataset

Dataset({
    features: ['output', 'input', 'instruction', '__index_level_0__', 'text'],
    num_rows: 41408
})

# Finetune model

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments


# Define the training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=100,
    num_train_epochs=1,
    # evaluation_strategy="steps",
    # save_strategy="steps",
    # save_total_limit=1,
    # load_best_model_at_end=True,
    # metric_for_best_model="eval_loss",
    # greater_is_better=False,
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
)

# Initialize Trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    # eval_dataset=test_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=training_args,
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/41408 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [7]:
# Finetuning the model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 41,408 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 100
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.722700
2,1.627200
3,1.663200
4,1.993800
5,1.383100
6,1.328000
7,1.351800
8,1.079400
9,1.069500
10,1.147200


In [8]:
# trainer.evaluate()

# Model evaluation

In [9]:
FastLanguageModel.for_inference(model)

# Function to generate prompts and get response
def generate_response(instruction, input):
  inputs = tokenizer(
  [
      alpaca_prompt.format(
          instruction, # instruction
          input, # input
          "", # output
      )
  ], return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
  return tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Evaluate model using random prompt from test_dataset

In [31]:
from nltk.translate.bleu_score import corpus_bleu

# Sample
test_dataset = test_dataset.shuffle(seed=42).select(range(100))

# Generate responses for the sampled entry
generated_response = generate_response(test_dataset["instruction"][0], test_dataset["input"][0])

# Prepare reference response
reference_response = test_dataset["output"]

# Print evaluated sample prompts and outputs
for idx, (instr, inp, outp, gen_resp) in enumerate(zip(test_dataset["instruction"], test_dataset["input"], reference_response, generated_response), 1):
    print("--------------------------------------------------------------")
    print(f"Original Response {idx}:")
    print("\n### Instruction:\n", instr.strip())
    print("\n### Input:\n", inp.strip())
    print("\n### Expected Output:\n", outp.strip())
    print("--------------------------------------------------------------")
    print("Generated Response:\n", gen_resp.strip())

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


--------------------------------------------------------------
Original Response 1:

### Instruction:
 Generate a new summary of the passage below.

### Input:
 Basketball is a team sport in which two teams, usually consisting of five players on each team, play against each other on a rectangular court. The objective is to score by shooting the ball through a hoop 18 inches in diameter affixed to a backboard at each end of the court.

### Expected Output:
 Basketball involves two teams of typically five players each, playing on a rectangular court with the goal of scoring by shooting a ball through an 18-inch hoop attached to a backboard on either end.
--------------------------------------------------------------
Generated Response:
 Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate a new summary of the passage below.

### Input:
Basketball is a team s

# Evaluate model using promt

In [11]:
# Evaluate model using prompts
instruction = "List the prime numbers contained within the range."
input = "1-50"

response = generate_response(instruction, input)
print("Generated Response:")
print("\n".join(response))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generated Response:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
List the prime numbers contained within the range.

### Input:
1-50

### Response:
The prime numbers contained within the range 1-50 are 2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, and 47.


# Saving finetuned model


In [12]:
# Save finetuned model
model.save_pretrained("finetuned_model")